In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from tqdm import tqdm

# Data Analysis

In [ ]:
data = pd.read_csv("test.csv",engine = 'python')

In [ ]:
data.tail()

In [ ]:
colum = data.columns
for i in colum:
    print(f'{len(set(data[i]))} different values in the {i} column')
print(f"\ntotal number of examples {len(data)}")

#Host, link, Time(ET), Time(GMT),is of no use for trainig the function

In [ ]:
data = data.drop(["Host", "Link", "Date(ET)", "Time(ET)", "time(GMT)"], axis=1)

In [ ]:
colum = data.columns
for i in colum:
    print(f'{len(set(data[i]))} different values in the {i} column')
print(f"\ntotal number of examples {len(data)}")



In [ ]:
list(set(data["Source"])) # differnet values in "Source" column

In [ ]:
# repalcing FACEBOOK to Facebook 
data.replace(to_replace='FACEBOOK', value='Facebook',inplace=True)
# Now there are only 4 different values in "Source" column

In [ ]:
Counter(data.loc[:,"Source"]) #  # distribution of differnet values in "Source" column

In [ ]:
Counter(data.iloc[:,[-1]]['Patient_Tag']) # distribution of labels in the "Patien_Tag" column
# It's an unbalanced data

In [ ]:
dummy = {}
for i in list(set(data["Source"])):
    print(i,"---", Counter(data.iloc[:,[0,-1]][data['Source'] == i]['Patient_Tag']))
# distribution of labels with reference to each values in "Source" column

In [ ]:
replace_ = {}
for index, i in enumerate(list(set(data["Source"])),start=1):
    replace_[index] = i
    data.replace(to_replace=i, value=index,inplace=True)
data.fillna('UNK',inplace=True)

In [ ]:
list(set(data["Source"]))

In [ ]:
data.fillna('UNK',inplace=True)

# Vocab creation

In [ ]:
import re

In [ ]:
rep_with = ['.', '?', '/', '\n', '(', ')','[', ']', '{', '}', '-','"','!', '|' ]

In [ ]:
def rep_(sent):
    for i in rep_with:
        sent = sent.replace(i,' ').replace('$', ' ').replace(',','').replace("'",'')
    return sent

In [ ]:
import re
import num2words

def n2w(text):
    return re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)

In [ ]:
def preprocess(data,pos):
    sent = []
    for i in range(len(data)):
        try:sent.append(n2w(rep_(data.iloc[i,pos])))
        except:print(data.iloc[i,pos])
    return sent

In [ ]:
sent = preprocess(data, 2)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
sent_preprocess = []
for i in sent:
    sent_preprocess.append([w for w in word_tokenize(i) if not w in stop_words])
sent = sent_preprocess
# sent = [i.split(' ') for i in sent]

In [ ]:
train = [[sent[i], data.loc[:,'Patient_Tag'][i]] for i in range(len(sent))]
train = sorted(train, key = lambda c: len(c[0])) # sorted train using the len of x {helps in minibatching}

In [ ]:
train.pop(0)

In [ ]:
train_len = []  # len of each example
for i in train:
    train_len.append(len(i[0]))

In [ ]:
import pickle
try: 
    with open("elmo_st.pkl", "rb") as f:
        print("loading embeddings")
        embeddings = pickle.load(f)
    
except:
    print("creating embeddings ...it will take time")
    from allennlp.commands.elmo import ElmoEmbedder

    elmo = ElmoEmbedder(
    options_file='https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json', 
    weight_file='https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5',
    cuda_device=1)

    #define max token length, 2187 is the max
    max_tokens=1024
    #input sentences
    sentences = []
#     x = [i[0] for i in train]
    for i in range(0,len(sent),16):
        sentences.append(sent[i:i+16])
    #create a pretrained elmo model (requires internet connection)
    embeddings=[]

    #loop through the input sentences
    for k,j in enumerate(sentences):
        print(k)
        for i, elmo_embedding in enumerate(elmo.embed_sentences(j)): 
            # Average the 3 layers returned from Elmo
            avg_elmo_embedding = np.average(elmo_embedding, axis=0)
            padding_length = max_tokens - avg_elmo_embedding.shape[0]
            if(padding_length>0):
                avg_elmo_embedding =np.append(avg_elmo_embedding, np.zeros((padding_length, avg_elmo_embedding.shape[1])), axis=0)
            else:
                avg_elmo_embedding=avg_elmo_embedding[:max_tokens]
            embeddings.append(avg_elmo_embedding)

In [ ]:
with open("test_elmo.pkl", "wb") as f:
        pickle.dump(embeddings,f)

In [ ]:
len(embeddings)

# Training

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0")

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, num_layers,directions,bidirectonal,out):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.directions = directions
        self.gru = nn.GRU(1024, hidden_size, num_layers,bidirectional=bidirectonal)
        self.linear1 = nn.Linear(hidden_size*directions, 32)
        self.linear2 = nn.Linear(32, out, bias = True)
        self.drop = nn.Dropout(p=0.7, inplace=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inp, hidden):
        out,h = self.gru(inp,hidden)
        out = self.linear1(out)#.view(-1,2,out.shape[2]))
        out = self.linear2(self.drop(out))
        return self.sigmoid(out)

    def init_hidden(self, batch_size):
        print()
        return torch.zeros(self.num_layers*self.directions, batch_size, self.hidden_size, dtype=torch.double)

In [ ]:
model = EncoderRNN(hidden_size=256,num_layers=1,directions=1,bidirectonal=False,out=1)

In [ ]:
model.to(device).double()

In [ ]:
y = [i[1] for i in train]

In [ ]:
import pickle

In [ ]:
with open("out.pkl", "wb") as f:
    pickle.dump(y,f)

In [ ]:
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
batch = 64
for epoch in range(5):
    running_loss = 0
    for i in range(0,len(embeddings),batch):
        h = model.init_hidden(len(embeddings[i:i+batch])).to(device)
        target = torch.tensor(y[i:i+batch],dtype=torch.double).to(device)
        inp = torch.tensor(embeddings[i:i+batch],dtype=torch.double).view(1830,len(embeddings[i:i+batch]),1024).to(device)
        out = model(inp,h)
        loss = loss_function(out[-1,:,:],target)
        nn.utils.clip_grad_norm_(model.parameters(), 50)
        running_loss+=loss.item()
        print(loss.item())
        loss.backward()
        optimizer.step()
    print("loss", running_loss/len(embeddings))